In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/NLP

/content/drive/MyDrive/NLP


In [3]:
import pandas as pd
import numpy as np
# import torch
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('AlexaCombinedReviews.csv')
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,\n\nI'm so fond of my Echo!
1,Loved it!,\n\nI adored it!
2,"Sometimes while playing a game, you can answer...","\n\nSometimes when playing a game, I can answe..."
3,I have had a lot of fun with this thing. My 4 ...,\n\nI've had a blast with this device. My four...
4,Music,is the food of the soul\n\nMusic nourishes th...


In [5]:
df.shape

(3150, 2)

In [6]:
df.isnull().sum()

verified_reviews       0
paraphrased_reviews    1
dtype: int64

In [7]:
mask = df.isnull()
row_has_missing = mask.any(axis=1)
print("Row indices with missing values:")
print(df.index[row_has_missing].tolist())

Row indices with missing values:
[434]


In [8]:
df.dropna(inplace=True)

In [9]:
df.shape

(3149, 2)

In [10]:
import re
# define a regular expression to match any newline character
newline_regex = re.compile(r'\r?\n')

# define a lambda function to remove any newline characters from a string
remove_newlines = lambda s: newline_regex.sub('', s)

# apply the lambda function to the 'paraphrased_reviews' column
df['paraphrased_reviews'] = df['paraphrased_reviews'].apply(remove_newlines)

In [11]:
# for i in df['paraphrased_reviews']:
#     if '\n' in i:
#        i=i.replace('\n','')

In [12]:
# def remove_char(remove_newlines):
#     remove_newlines = lambda x: x.replace('\n', '')
#     return remove_newlines

In [13]:
# df['paraphrased_reviews'] = df['paraphrased_reviews'].replace(remove_char)

In [14]:
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,I'm so fond of my Echo!
1,Loved it!,I adored it!
2,"Sometimes while playing a game, you can answer...","Sometimes when playing a game, I can answer a ..."
3,I have had a lot of fun with this thing. My 4 ...,I've had a blast with this device. My four yea...
4,Music,is the food of the soulMusic nourishes the sp...


In [15]:
count=0
for i in df['verified_reviews']:
    if len(i)==1:
        count+=1
        # print(i)
print(count)



81


In [16]:
df = df[~(df['verified_reviews'].str.len() == 1)]

In [17]:
df.head()

,verified_reviews,paraphrased_reviews
0,Love my Echo!,I'm so fond of my Echo!
1,Loved it!,I adored it!
2,"Sometimes while playing a game, you can answer...","Sometimes when playing a game, I can answer a ..."
3,I have had a lot of fun with this thing. My 4 ...,I've had a blast with this device. My four yea...
4,Music,is the food of the soulMusic nourishes the sp...


In [18]:
df['verified_reviews'].isnull().sum()

0

In [19]:
df = df[~(df['verified_reviews'].str.split(" ").str.len() == 1)]

In [20]:
df.shape

(2997, 2)

In [21]:
df.tail()

,verified_reviews,paraphrased_reviews
3144,love it,or leave itTake it or leave it.
3145,"Perfect for kids, adults and everyone in betwe...",Great for all ages!
3146,"Listening to music, searching locations, check...","There are many more tasks that can be learned,..."
3147,"I do love these things, i have them running my...",I'm really fond of these gadgets; they control...
3148,Only complaint I have is that the sound qualit...,My only issue with the dot is the sound qualit...


In [22]:
emoticons = re.compile('[:;=]-?[)(DP]')

df['verified_reviews'] = df['verified_reviews'].replace(emoticons, '')

# print(df["verified_reviews"][19])

In [23]:
# from transformers import GPT2Tokenizer
# model_name="gpt2"


In [24]:
dataframe = pd.DataFrame()
for row in df.itertuples():
  dataframe = pd.concat([dataframe, pd.DataFrame.from_records([{ 'reviews': row.verified_reviews, "isHuman": 1}])], ignore_index=True)
  dataframe = pd.concat([dataframe, pd.DataFrame.from_records([{ 'reviews': row.paraphrased_reviews, "isHuman": 0}])], ignore_index=True)
dataframe.head()

,reviews,isHuman
0,Love my Echo!,1
1,I'm so fond of my Echo!,0
2,Loved it!,1
3,I adored it!,0
4,"Sometimes while playing a game, you can answer...",1


In [25]:
labels = dataframe["isHuman"]
training_text = dataframe["reviews"].tolist()
print(labels[0])

1


In [26]:
# pip install transformers

In [27]:
from transformers import AutoTokenizer, GPT2Config, GPT2Model, AdamW, get_linear_schedule_with_warmup
import torch
from progressbar import progressbar
from csv import writer
from keras.utils import pad_sequences
from sklearn import preprocessing

def get_device():
  if torch.cuda.is_available():
      device = 'cuda:0'
  else:
      device = 'cpu'
  return device

def tokenize(training_text):
  encodedSentences = []
  tokenizer = AutoTokenizer.from_pretrained('gpt2')
  for data in training_text:
    encoded_input = tokenizer.encode(data, add_special_tokens=True)
    encodedSentences.append(encoded_input)
  return encodedSentences, pad_sequences(encodedSentences, maxlen=512, dtype="long", value=0, truncating="post", padding="post")

def attentionMask(encodedText):
  sentences_attention_masks = []
  for encoding in encodedText:
      att_mask = [int(token_id > 0) for token_id in encoding]
      sentences_attention_masks.append(att_mask)
  return sentences_attention_masks

def label_preprocessing(label_data,convert = True):
  le = preprocessing.LabelEncoder()
  if convert :
      labels = le.fit_transform(label_data)
      return labels
  else:
      labels = le.inverse_transform(label_data)
      return labels

def flat_accuracy(preds, labels):
  return (preds == labels).mean()

tokenize_text, tokenize_text_pad = tokenize(training_text)
labels = label_preprocessing(labels)
# print(tokenize_text[0])
sentences_attention_masks = attentionMask(tokenize_text_pad)
print(sentences_attention_masks)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [28]:
print(tokenize_text[0])

[18565, 616, 21455, 0]


In [29]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(tokenize_text_pad, labels,
                                                                                    random_state=2018, test_size=0.1)
train_masks, test_masks, _, _ = train_test_split(sentences_attention_masks, labels,
                                                        random_state=2018, test_size=0.1)

In [30]:
len(train_masks[0])

512

In [31]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_inputs = torch.tensor(train_inputs)
test_inputs = torch.tensor(test_inputs)

train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_masks)
test_masks = torch.tensor(test_masks)


# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=8)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=8)

In [32]:
model = GPT2Model.from_pretrained('gpt2',
                                num_labels=2,
                                output_attentions=False,
                                output_hidden_states=False
                                ).to(get_device())

optimizer = AdamW(model.parameters(),
                      lr=2e-5,  # args.learning_rate - default is 5e-5, our notebook had 2e-5
                      eps=1e-8
                      )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [33]:
import random
import time

def start_training(epochs, optimizer, model, train_dataloader, test_dataloader):
  torch.cuda.empty_cache()
  total_steps = len(train_dataloader) * epochs

  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps=0,  # Default value in run_glue.py
                                              num_training_steps=total_steps)

  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  loss_values = []
  device = get_device()

  for epoch_i in range(0, epochs):

      # ========================================
      #               Training
      # ========================================

      print("")
      print(
          '======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      t0 = time.time()

      total_loss = 0

      model.train()

      # For each batch of training data...
      for step, batch in progressbar(enumerate(train_dataloader)):
          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              # elapsed = self.__format_time__(time.time() - t0)

              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.'.format(
                  step, len(train_dataloader)))

          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          model.zero_grad()

          outputs = model(b_input_ids,
                          token_type_ids=None,
                          attention_mask=b_input_mask)#,
                          #labels=b_labels)
          loss = outputs[0]
          print(loss)
          print(loss.shape)
          print(loss[0].shape)

          total_loss += loss.item()

          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          optimizer.step()
          scheduler.step()
      avg_train_loss = total_loss / len(train_dataloader)
      loss_values.append(avg_train_loss)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      # print("  Training epcoh took: {:}".format(
      #     self.__format_time__(time.time() - t0)))
      print("")
      print("Running Validation...")
      # t0 = time.time()
      model.eval()

      # Tracking variables
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      for batch in test_dataloader:
          batch = tuple(t.to(device) for t in batch)
          b_input_ids, b_input_mask, b_labels = batch
          with torch.no_grad():
              outputs = model(b_input_ids,
                              token_type_ids=None,
                              attention_mask=b_input_mask)
          logits = outputs[0]
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()
          tmp_eval_accuracy = flat_accuracy(logits, label_ids)
          eval_accuracy += tmp_eval_accuracy
          nb_eval_steps += 1
          print("  Accuracy: {0:.2f}".format(
              eval_accuracy/nb_eval_steps))

  print("")
  print("Training complete!")

In [34]:
torch.cuda.empty_cache()
start_training(5, optimizer, model, train_dataloader, test_dataloader)

/ |#                                                  | 0 Elapsed Time: 0:00:00


======== Epoch 1 / 5 ========
Training...
tensor([[[ 9.4589e-03, -7.7310e-02,  4.7861e-01,  ...,  1.4147e-01,
          -5.8280e-02,  1.9994e-01],
         [-3.0976e-01, -6.1792e-01, -7.0628e-01,  ...,  2.7664e-01,
          -1.0654e-01,  2.9180e-01],
         [ 2.2315e-01, -1.8579e-01, -9.6592e-01,  ..., -1.2104e-01,
          -5.0817e-01,  6.9808e-01],
         ...,
         [ 9.4846e-02, -5.1787e-01, -3.5538e-01,  ..., -1.3622e-01,
          -2.3214e-01, -1.0584e-01],
         [ 9.0099e-02, -3.0698e-01, -2.7457e-01,  ..., -3.7087e-01,
          -2.4739e-01, -4.6608e-02],
         [-6.5139e-02, -3.3304e-01, -2.6345e-01,  ..., -1.2734e-01,
          -2.9192e-01, -1.1338e-01]],

        [[ 6.7790e-02, -9.2249e-02,  4.7648e-01,  ...,  1.2614e-01,
          -3.9228e-01,  5.1486e-02],
         [-3.8320e-01, -3.5894e-01, -4.1859e-01,  ...,  2.1880e-01,
           1.3745e-01,  3.8952e-02],
         [-8.2237e-02,  2.6777e-01, -1.4996e+00,  ...,  5.0166e-01,
          -5.3396e-02,  2.0092e-0

RuntimeError: ignored

In [ ]:
import torch.nn.functional as F

# Flatten the predictions and true labels
logits_flat = outputs.view(-1, outputs.shape[-1])
labels_flat = b_labels.view(-1)

# Calculate the cross-entropy loss
loss = F.cross_entropy(logits_flat, labels_flat)

In [ ]:
# # dataframe['reviews'] = dataframe['reviews'].apply(lambda x: tokenizer(x, return_tensors='pt'))
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
# model = GPT2Model.from_pretrained('gpt2')
# # torch.device(device)
# # dataframe['reviews'] = dataframe['reviews'].apply(lambda x: model(x))

# # Define a function to embed a sentence using GPT-2
# def embed_sentence_gpt2(sentence):
#   # print(sentence)
#   # Tokenize the sentence
#   encoded_input = tokenizer(sentence, return_tensors='pt')
#   # Get the model's hidden state for the sentence
#   with torch.no_grad():
#     output = model(**encoded_input)
#   embeddings = output.last_hidden_state.mean(dim=1).numpy()
#   return embeddings

# # # Example usage:
# # # sentence = "This is a test sentence."
# # # gpt2_embedding = embed_sentence_gpt2(sentence)
# # # df['verified_reviews'] = df['verified_reviews'] + tokenizer.eos_token
# # # df['verified_reviews'][0]

# # print(df)
# tempList = []
# dfTemp = pd.DataFrame()
# with open('embedding_gen.csv', 'a') as f_object:
#     writer_object = writer(f_object)
#     writer_object.writerow(["embedding"])
#     f_object.close()
# # print(tempList)
# for review in progressbar(dataframe['reviews']):
#   embedding = embed_sentence_gpt2(review)
#   with open('embedding_gen.csv', 'a') as f_object:
#         writer_object = writer(f_object)
#         writer_object.writerow(embedding)
#         f_object.close()

# tempList


# # Tokenize text column
# # dataframe['reviews'] = dataframe['reviews'].apply(lambda x: embed_sentence_gpt2(x))
# # df['paraphrased_reviews'] = df['paraphrased_reviews'].apply(lambda x: embed_sentence_gpt2(x))

# # # Print updated data frame
# # print(df)


In [ ]:
# x_train, y_train,x_test,y_test = train_test_split(df['reviews'], df["isHuman"], test_size=0.2, random_state=265)
# print(x_train)

In [ ]:
# c = 1048576
# print(0x80 | (c >> 21))
# # print(0x80 | (c & 0x3F))
# # print(0xe0 | (c >> 18))
# # print(0x80 | (c & 0x3F))